In [0]:
import pyspark
from pyspark.sql import SparkSession
import json
from urllib.request import urlopen
from urllib.error import HTTPError

In [0]:
dbutils.widgets.text('filepath','https://config4.blob.core.windows.net/config/config3.json')

In [0]:
def readingConfigFile(filepath):
    """ This function reads the configuration file from ADLS """
    try:
        file = urlopen(filepath)
    except HTTPError:
        print("Incorrect URL.")
        return
    data = ""
    for line in file: data += line.decode()
    try:
        config = json.loads(data)
        return config
    except ValueError as e:
        return "Decoding JSON file failed"

In [0]:
config = readingConfigFile(dbutils.widgets.get('filepath'))

In [0]:
config

Out[60]: {'key': 'newstorage512',
 'scope': 'testscope',
 'container': 'adventureworks',
 'storageAccountName': 'newstorage512',
 'sourceDir': 'bronzezone',
 'sinkDir': 'silverzone',
 'tablenames': ['Address',
 'Customer',
 'CustomerAddress',
 'Product',
 'ProductCategory',
 'ProductDescription',
 'ProductModel',
 'ProductModelProductDescription',
 'SalesOrderDetail',
 'SalesOrderHeader'],
 'dropColumns': {'Address': ['rowguid'],
 'Customer': ['NameStyle', 'rowguid'],
 'CustomerAddress': ['rowguid'],
 'Product': ['DiscontinuedDate', 'rowguid'],
 'ProductCategory': ['rowguid'],
 'ProductDescription': ['rowguid'],
 'ProductModel': ['rowguid'],
 'ProductModelProductDescription': ['rowguid'],
 'SalesOrderDetail': ['rowguid'],
 'SalesOrderHeader': ['RevisionNumber',
 'Status',
 'OnlineOrderFlag',
 'CreditCardApprovalCode',
 'Comment',
 'rowguid']},
 'nullHandling': {'Address': ['AddressLine2'],
 'Customer': ['Title', 'MiddleName', 'Suffix'],
 'CustomerAddress': [],
 'Product': ['Weight', 'SellEndDate', 'Size', 'Color'],
 'ProductCategory': ['ParentProductCategoryID'],
 'ProductDescription': [],
 'ProductModel': ['CatalogDescription'],
 'ProductModelProductDescription': [],
 'SalesOrderDetail': [],
 'SalesOrderHeader': []},
 'replaceNullValue': {'string': 'NA',
 'timestamp': '1900-01-01 00:00:00',
 'boolean': False,
 'numeric': -1}}

In [0]:
spark.conf.set(
    "fs.azure.account.key."+config["storageAccountName"]+".dfs.core.windows.net",
    dbutils.secrets.get(scope=config["scope"],key=config["key"])) 

In [0]:
SalesLT = {}

for table in config['tablenames']:
    try:
        SalesLT[table] = spark.read.format("parquet").option("header","true").load("abfss://"+config["container"]+"@"+config["storageAccountName"]+".dfs.core.windows.net/incremental/"+table+"_part.parquet")
    except Exception as e:
        print(e)

In [0]:
from pyspark.sql import functions as f

print("BEFORE CASTING")
SalesLT["SalesOrderDetail"].printSchema()
SalesLT["SalesOrderDetail"] = SalesLT["SalesOrderDetail"].withColumn("SalesOrderID",f.col("SalesOrderID").cast("bigint"))
print("AFTER CASTING")
SalesLT["SalesOrderDetail"].printSchema()

BEFORE CASTING
root
-- SalesOrderID: integer (nullable = true)
-- SalesOrderDetailID: integer (nullable = true)
-- OrderQty: integer (nullable = true)
-- ProductID: integer (nullable = true)
-- UnitPrice: decimal(19,4) (nullable = true)
-- UnitPriceDiscount: decimal(19,4) (nullable = true)
-- LineTotal: decimal(38,6) (nullable = true)
-- rowguid: string (nullable = true)
-- ModifiedDate: timestamp (nullable = true)

AFTER CASTING
root
-- SalesOrderID: long (nullable = true)
-- SalesOrderDetailID: integer (nullable = true)
-- OrderQty: integer (nullable = true)
-- ProductID: integer (nullable = true)
-- UnitPrice: decimal(19,4) (nullable = true)
-- UnitPriceDiscount: decimal(19,4) (nullable = true)
-- LineTotal: decimal(38,6) (nullable = true)
-- rowguid: string (nullable = true)
-- ModifiedDate: timestamp (nullable = true)

In [0]:
SalesLT_ColumnTypes = {}

for table in config['tablenames']:
    #storing column types of each table to handle null
    SalesLT_ColumnTypes[table] = dict(SalesLT[table].dtypes)

In [0]:
def handleNull(DF,cols,colTypes):
    """
        Args:
            DF(Spark.DataFrame): DataFrame Object on which analysis should be performed
            cols(list): list of column names that has to be null handled
            colTypes(dictionary): dictionary of column name and its data types
        Returns:
            Spark.DataFrame: Null handled DataFrame Object
    """
    for col in cols:
        if colTypes[col] == "string":
            DF = DF.na.fill({col:config["replaceNullValue"]["string"]})
        elif colTypes[col] == "timestamp":
            DF = DF.na.fill({col:config["replaceNullValue"]['timestamp']})
        elif colTypes[col] == "boolean":
            DF = DF.na.fill({col:config["replaceNullValue"]['boolean']})
        else:
            DF = DF.na.fill({col:config["replaceNullValue"]['numeric']})
    return DF

In [0]:
def dropColumns(tablename,df,cols):
    """
        Args:
            tablename: Name of the DataFrame Object Table
            DF(Spark.DataFrame): DataFrame Object on which analysis should be performed
            cols(list): list of column names that has to be dropped
        Returns:
            Spark.DataFrame: DataFrame Object without columns in 'cols'
    """
    for col in cols:
        df=df.drop(col)
    return df

In [0]:
for table in config['tablenames']:
    SalesLT[table] = handleNull(SalesLT[table],config["nullHandling"][table],SalesLT_ColumnTypes[table])

In [0]:
for table in config['tablenames']:
    SalesLT[table]= dropColumns(table,SalesLT[table],config["dropColumns"][table])

In [0]:
SalesLT["Product"]=SalesLT["Product"].withColumn("Size",
                   f.when((SalesLT["Product"].Size>=38) & (SalesLT["Product"].Size<=42), 'S')
                   .when((SalesLT["Product"].Size>=44) & (SalesLT["Product"].Size<=48), 'M')
                   .when((SalesLT["Product"].Size>=50) & (SalesLT["Product"].Size<=54), 'L')
                   .when((SalesLT["Product"].Size>=56) & (SalesLT["Product"].Size<=62), 'XL')
                   .otherwise(SalesLT["Product"].Size))

SalesLT["Product"].show()

+---------+--------------------+-------------+------------+------------+---------+----+------+-----------------+--------------+-------------------+-------------------+--------------------+----------------------+--------------------+
ProductID| Name|ProductNumber| Color|StandardCost|ListPrice|Size|Weight|ProductCategoryID|ProductModelID| SellStartDate| SellEndDate| ThumbNailPhoto|ThumbnailPhotoFileName| ModifiedDate|
+---------+--------------------+-------------+------------+------------+---------+----+------+-----------------+--------------+-------------------+-------------------+--------------------+----------------------+--------------------+
 940| HL Road Pedal| PD-R853|Silver/Black| 35.9596| 80.9900| NA|149.00| 17| 70|2007-07-01 00:00:00|1900-01-01 00:00:00|[47 49 46 38 39 6...| clipless_pedals_s...|2008-03-11 10:03:...|
 714|Long-Sleeve Logo ...| LJ-0192-M| Multi| 38.4923| 49.9900| M| -1.00| 25| 11|2005-07-01 00:00:00|1900-01-01 00:00:00|[47 49 46 38 39 6...| awc_jersey_male_s...|2008-03-11 10:01:...|
 713|Long-Sleeve Logo ...| LJ-0192-S| Multi| 38.4923| 49.9900| S| -1.00| 25| 11|2005-07-01 00:00:00|1900-01-01 00:00:00|[47 49 46 38 39 6...| awc_jersey_male_s...|2008-03-11 10:01:...|
 712| AWC Logo Cap| CA-1098| Multi| 6.9223| 8.9900| NA| -1.00| 23| 2|2005-07-01 00:00:00|1900-01-01 00:00:00|[47 49 46 38 39 6...| no_image_availabl...|2008-03-11 10:01:...|
 711|Sport-100 Helmet,...| HL-U509-B| Blue| 13.0863| 34.9900| NA| -1.00| 35| 33|2005-07-01 00:00:00|1900-01-01 00:00:00|[47 49 46 38 39 6...| no_image_availabl...|2008-03-11 10:01:...|
 710|Mountain Bike Soc...| SO-B909-L| White| 3.3963| 9.5000| L| -1.00| 27| 18|2005-07-01 00:00:00|2006-06-30 00:00:00|[47 49 46 38 39 6...| no_image_availabl...|2008-03-11 10:01:...|
 709|Mountain Bike Soc...| SO-B909-M| White| 3.3963| 9.5000| M| -1.00| 27| 18|2005-07-01 00:00:00|2006-06-30 00:00:00|[47 49 46 38 39 6...| no_image_availabl...|2008-03-11 10:01:...|
 708|Sport-100 Helmet,...| HL-U509| Black| 13.0863| 34.9900| NA| -1.00| 35| 33|2005-07-01 00:00:00|1900-01-01 00:00:00|[47 49 46 38 39 6...| no_image_availabl...|2008-03-11 10:01:...|
 707|Sport-100 Helmet,...| HL-U509-R| Red| 13.0863| 34.9900| NA| -1.00| 35| 33|2005-07-01 00:00:00|1900-01-01 00:00:00|[47 49 46 38 39 6...| no_image_availabl...|2008-03-11 10:01:...|
+---------+--------------------+-------------+------------+------------+---------+----+------+-----------------+--------------+-------------------+-------------------+--------------------+----------------------+--------------------+

In [0]:
#Writing All Tables into Silverzone
for table in config['tablenames']:
    try:
        SalesLT[table].write.format("parquet").option("header","true").mode("append").save("abfss://"+config["container"]+"@"+config["storageAccountName"]+".dfs.core.windows.net/"+config["sinkDir"]+"/"+table)
    except Exception as e:
        print(e)
    

In [0]:
SalesLT = {}

for table in config['tablenames']:
    try:
        SalesLT[table] = spark.read.format("parquet").option("header","true").load("abfss://"+config["container"]+"@"+config["storageAccountName"]+".dfs.core.windows.net/"+config["sinkDir"]+"/"+table)
    except Exception as e:
        print(e)

In [0]:
#Function for profiling

import numpy
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, when, count, col

def dataprofile(data_all_df):
    data_cols = data_all_df.columns
    data_df = data_all_df.select(data_cols)
    columns2Bprofiled = data_df.columns
    
    global schema_name, table_name
    if not 'schema_name' in globals():
        schema_name = 'schema_name'
    if not 'table_name' in globals():
        table_name = 'table_name' 
    dprof_df = pd.DataFrame({'column_names':data_df.columns,\
                             'data_types':[x[1] for x in data_df.dtypes]}) 
    dprof_df = dprof_df[['column_names', 'data_types']]
    
    # ======================
    
    num_rows = data_df.count()
    dprof_df['num_rows'] = num_rows
    
    # ======================    
    
    # number of rows with nulls and nans   
    df_nacounts = data_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data_df.columns \
                                  if data_df.select(c).dtypes[0][1]!='timestamp' and data_df.select(c).dtypes[0][1]!='binary' and data_df.select(c).dtypes[0][1]!='boolean']).toPandas().transpose()
    df_nacounts = df_nacounts.reset_index()  
    df_nacounts.columns = ['column_names','num_null']
    dprof_df = pd.merge(dprof_df, df_nacounts, on = ['column_names'], how = 'left')
    
    # ========================
    
    # number of rows with white spaces (one or more space) or blanks
    num_spaces = [data_df.where(F.col(c).rlike('^\\s+$')).count() for c in data_df.columns]
    dprof_df['num_spaces'] = num_spaces
    num_blank = [data_df.where(F.col(c)=='').count() for c in data_df.columns]
    dprof_df['num_blank'] = num_blank
    
    # =========================
    
    # using the in built describe() function 
    desc_df = data_df.describe().toPandas().transpose()
    desc_df.columns = ['count', 'mean', 'stddev', 'min', 'max']
    desc_df = desc_df.iloc[1:,:]  
    desc_df = desc_df.reset_index()  
    desc_df.columns.values[0] = 'column_names'  
    desc_df = desc_df[['column_names','count', 'mean', 'stddev']] 
    dprof_df = pd.merge(dprof_df, desc_df , on = ['column_names'], how = 'left')
    
    # ===========================================    
    
    df_counts = dprof_df[['column_names']]
    dprof_df = pd.merge(dprof_df, df_counts , on = ['column_names'], how = 'left')  
    
    # ==========================================
    
    # number of distinct values in each column
    dprof_df['num_distinct'] = [data_df.select(x).distinct().count() for x in columns2Bprofiled]
    
    # ============================================
    
    # most frequently occuring value in a column and its count
    dprof_df['most_freq_valwcount'] = [data_df.groupBy(x).count().sort("count",ascending=False).limit(1).\
                                       toPandas().iloc[0].values.tolist() for x in columns2Bprofiled]
    dprof_df['most_freq_value'] = [x[0] for x in dprof_df['most_freq_valwcount']]
    dprof_df['most_freq_value_count'] = [x[1] for x in dprof_df['most_freq_valwcount']]
    dprof_df = dprof_df.drop(['most_freq_valwcount'],axis=1)
    
    # ============================================
    
    # least frequently occuring value in a column and its count
    dprof_df['least_freq_valwcount'] = [data_df.groupBy(x).count().sort("count",ascending=True).limit(1).\
                                        toPandas().iloc[0].values.tolist() for x in columns2Bprofiled]
    dprof_df['least_freq_value'] = [x[0] for x in dprof_df['least_freq_valwcount']]
    dprof_df['least_freq_value_count'] = [x[1] for x in dprof_df['least_freq_valwcount']]
    dprof_df = dprof_df.drop(['least_freq_valwcount'],axis=1)

    return dprof_df


In [0]:
from datetime import datetime

now = datetime.now()
date_time = now.strftime("%m-%d-%Y, %H:%M:%S")
#With each run, new profile statistics will be created for each table in a new folder using date_time as folder name

for table in config['tablenames']:
    result = dataprofile(SalesLT[table]) 
    resultDF = spark.createDataFrame(result.astype(str))
    resultDF.repartition(1).write.option('header',True).mode('overwrite').format('csv').save("abfss://"+config["container"]+"@"+config["storageAccountName"]+".dfs.core.windows.net/"+"Profile Statistics"+"/"+table+"/"+date_time)


/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [StandardCost] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(
/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [ListPrice] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(
/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [Weight] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(
/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [UnitPrice] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(
/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [UnitPriceDiscount] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(
/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [LineTotal] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(
/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [SubTotal] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(
/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [TaxAmt] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(
/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [Freight] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(
/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [TotalDue] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(